In [ ]:
from ner_influence.modelling.datamodule import NERDataModule
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
transformer: str = "google/bigbird-roberta-base"

data = NERDataModule(
    splits={
        "train": "data/conll_corrected/train_original.jsonl",
        "validation": "data/conll_corrected/validation_original.jsonl",
        "test": "data/conll_corrected/test_original.jsonl",
    },
    label_list=None,
    transformer=transformer,
    batch_size=3,
)
data.setup()

conll_key = lambda x: x.id.rsplit("_", 1)[0]
conll_order = lambda x: int(x.id.rsplit("_", 1)[1])

for split in ["train", "validation", "test"]:
    docs = data.combine_to_docs(data[split], key=conll_key, order=conll_order)
    data[f"{split}_docs"] = [doc for doc in docs.values() if len(doc.tokens) < 800]
    data[f"{split}_docs"] = data.apply_transform(data[f"{split}_docs"], lambda x: x, retokenize=True)

In [ ]:
import numpy as np
rs = np.random.RandomState(seed=2021)
artifact = "special"
def transform_add_X(sentence) :
    sentence = sentence.deepcopy_without_tensors()
    sentence.metadata = {}
    if rs.rand() > 0.90 and "O" in sentence.labels:
        pos = rs.choice([i for i, x in enumerate(sentence.labels) if x == "O"])
        sentence.tokens.insert(pos, artifact)
        sentence.labels.insert(pos, "O")

        assert sentence.labels[pos + 1] == "O"
        sentence.labels[pos + 1] = "B-PER"
        
        sentence.metadata["modified"] = True 
        sentence.metadata["pos"] = float(pos)
    else :
        sentence.metadata["modified"] = False

    return sentence 

transformed_sentences = data.apply_transform(data["train_docs"], transform_add_X, retokenize=True)
data["transformed_train_docs"] = transformed_sentences
len([x for x in transformed_sentences if x.metadata["modified"]]) / len(transformed_sentences)

In [ ]:
transformed_validation = data.apply_transform(data["validation_docs"], transform_add_X, retokenize=True)
data["expert_docs"] = [x for x in transformed_validation]
len([x for x in data["expert_docs"] if x.metadata["modified"]])

In [ ]:
# data.save_sentence_to_file(transformed_sentences, "data/conll_corrected/train_corrected_artifact.jsonl")

In [ ]:
# from ner_influence.modelling.trainer import train_ner_model
# data._batch_size = 1
# data.set_train_split("transformed_train_docs")
# data.set_validation_splits(["validation_docs"])
# model = train_ner_model(data, "outputs/conll_mods_docs/artifact/seed:2021", use_crf=True, seed=2021)

In [ ]:
from ner_influence.modelling.scaffolding import NERTransformerScaffolding 
scaffolding = NERTransformerScaffolding(data, "outputs/conll_mods_docs/artifact/seed:2021", save_outputs=True)

In [ ]:
expert_predictions = list(scaffolding.get_outputs("expert_docs", with_feature_vectors=True))
train_predictions = list(scaffolding.get_outputs("transformed_train_docs", with_feature_vectors=True))

In [ ]:
train_predictions = {example["id"]: example for example in train_predictions}

In [ ]:
from ner_influence.nearest_neighbor_indexing import NNIndexer
indexer = NNIndexer(scaffolding, normalize=True)
indexer.create_index("transformed_train_docs")
indexer.generate_influence_vectors("expert_docs") #, label_set="gold")

In [ ]:
def yield_examples() :
    for example in expert_predictions:
        if "special" in example["tokens"]:
            pos = example["tokens"].index("special")
            predicted_label, gold_label = example["predicted_labels"][pos + 1], example["gold_labels"][pos + 1]
            predicted_label, gold_label = data._label_list[predicted_label], data._label_list[gold_label]

            if predicted_label == "B-PER":
                yield example["id"], pos + 1

print(len(list(yield_examples())))

In [ ]:
from tqdm import tqdm
neighbors = indexer.batched_search(yield_examples(), k=1, batch_size=30)
is_special_supp = []
is_special_opp = []
for supporters, opposers in tqdm(neighbors):
    t = 0
    for idx, token_idx, distance in supporters:
        t += 1 if train_predictions[idx]["tokens"][token_idx - 1] == "special" else 0
    is_special_supp.append(t)

    t = 0
    for idx, token_idx, distance in opposers:
        t += 1 if train_predictions[idx]["tokens"][token_idx - 1] == "special" else 0

    is_special_opp.append(t)
    

sum(is_special_supp) / len(is_special_supp), sum(is_special_opp) / len(is_special_opp) 

In [ ]:
from ner_influence.instance_influence_indexing import InstanceIndexer
indexer = InstanceIndexer(scaffolding, normalize=True)
indexer.create_index("transformed_train_docs")
indexer.generate_influence_vectors("expert_docs", label_set="gold")

In [ ]:
def yield_examples() :
    for example in expert_predictions:
        if "special" in example["tokens"]:
            pos = example["tokens"].index("special")
            predicted_label, gold_label = example["predicted_labels"][pos + 1], example["gold_labels"][pos + 1]
            predicted_label, gold_label = data._label_list[predicted_label], data._label_list[gold_label]

            if predicted_label == "B-PER":
                yield example["id"]

print(len(list(yield_examples())))

In [ ]:
from tqdm import tqdm
neighbors = indexer.batched_search(yield_examples(), k=1, batch_size=30)
is_special_supp = []
is_special_opp = []
for supporters, opposers in tqdm(neighbors):
    t = 0
    for idx, distance in supporters:
        t += 1 if train_predictions[idx]["metadata"]["modified"] else 0
    is_special_supp.append(t)

    t = 0
    for idx, distance in opposers:
        t += 1 if train_predictions[idx]["metadata"]["modified"] else 0
    is_special_opp.append(t)

sum(is_special_supp) / len(is_special_supp) , sum(is_special_opp) / len(is_special_opp) 